In [ ]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.kill()

In [ ]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }
    
    response = requests.put(url, headers=headers, json=data)
    
    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

In [ ]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [ ]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [ ]:
from flask import Flask, request, abort, send_file
import logging
import os
from google.genai import types
from io import BytesIO
import uuid
import requests
from PIL import Image

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    ImageMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    ImageMessageContent
)

app = Flask(__name__)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
app.logger.setLevel(logging.INFO)


configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

# 儲存圖片的目錄
IMAGE_DIR = "/content/generated_images"
UPLOAD_DIR = "/content/uploaded_images"
os.makedirs(IMAGE_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)

# 產生圖片的函數
def generate_image(prompt):
    """使用 Gemini Imagen 產生圖片"""
    response = client.models.generate_images(
        model='imagen-4.0-generate-001',
        prompt=prompt,
        config=types.GenerateImagesConfig(
            number_of_images=1,
        )
    )
    # 取得第一張圖片
    generated_image = response.generated_images[0]
    
    # 儲存圖片（不使用 format 參數）
    image_filename = f"{uuid.uuid4()}.jpg"
    image_path = os.path.join(IMAGE_DIR, image_filename)
    generated_image.image.save(image_path)
    
    return image_filename

# 下載 LINE 圖片的函數
def download_line_image(message_id):
    """從 LINE 下載使用者上傳的圖片"""
    with ApiClient(configuration) as api_client:
        # 使用 MessagingApiBlob 來取得圖片內容
        line_bot_blob_api = MessagingApiBlob(api_client)
        image_content = line_bot_blob_api.get_message_content(message_id)
        
        # 儲存圖片（image_content 是 bytes 物件）
        image_filename = f"{uuid.uuid4()}.jpg"
        image_path = os.path.join(UPLOAD_DIR, image_filename)
        
        with open(image_path, 'wb') as f:
            f.write(image_content)
        
        return image_path

# 分析圖片的函數
def analyze_image(image_path):
    """使用 Gemini Vision API 分析圖片內容"""
    # 讀取圖片
    image = Image.open(image_path)
    
    # 使用 Gemini 分析圖片
    response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents=[
            "請用繁體中文台灣用語詳細說明這張圖片在幹嘛，包括圖片中的人物、物品、場景、動作等細節。",
            image
        ]
    )
    
    return response.text

@app.route("/images/<filename>")
def serve_image(filename):
    """提供圖片存取的路由"""
    image_path = os.path.join(IMAGE_DIR, filename)
    if os.path.exists(image_path):
        return send_file(image_path, mimetype='image/jpeg')
    else:
        abort(404)

@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            try:
                # 產生圖片
                image_filename = generate_image(prompt)
                
                # 取得 ngrok URL
                ngrok_url = tunnel.public_url
                image_url = f"{ngrok_url}/images/{image_filename}"
                
                # 回傳圖片訊息
                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[
                            TextMessage(text=f"正在為您產生圖片：{prompt}"),
                            ImageMessage(
                                original_content_url=image_url,
                                preview_image_url=image_url
                            )
                        ]
                    )
                )
            except Exception as e:
                # 如果產生圖片失敗，回傳錯誤訊息
                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=f"產生圖片時發生錯誤：{str(e)}")]
                    )
                )
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

@handler.add(MessageEvent, message=ImageMessageContent)
def handle_image_message(event):
    """處理使用者上傳的圖片"""
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        try:
            # 下載圖片
            image_path = download_line_image(event.message.id)
            
            # 使用 Gemini 分析圖片
            description = analyze_image(image_path)
            
            # 回傳分析結果
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=description)]
                )
            )
        except Exception as e:
            # 如果分析圖片失敗，回傳錯誤訊息
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"分析圖片時發生錯誤：{str(e)}")]
                )
            )

if __name__ == "__main__":
    app.run(port=port)